In [0]:
!kill -9 -1

In [0]:
import os

In [2]:
 !nvidia-smi -L

GPU 0: Tesla P100-PCIE-16GB (UUID: GPU-6383b2cd-8c9a-e579-1fd5-0603747dbd84)


In [3]:
# memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

  Created wheel for gputil: filename=GPUtil-1.4.0-cp36-none-any.whl size=7413 sha256=e4701d1f629e9db74adf3400b459c9b8a9b3dd54340805dc51b3c98c558435e5
  Stored in directory: /root/.cache/pip/wheels/3d/77/07/80562de4bb0786e5ea186911a2c831fdd0018bda69beab71fd
Successfully built gputil
Gen RAM Free: 26.4 GB  | Proc size: 158.4 MB
GPU RAM Free: 16280MB | Used: 0MB | Util   0% | Total 16280MB


In [4]:
!lscpu |grep 'Model name'

Model name:          Intel(R) Xeon(R) CPU @ 2.20GHz


In [5]:
#no.of sockets i.e available slots for physical processors
!lscpu | grep 'Socket(s):'

Socket(s):           1


In [6]:
#no.of cores each processor is having 
!lscpu | grep 'Core(s) per socket:'

Core(s) per socket:  2


In [7]:
#no.of threads each core is having
!lscpu | grep 'Thread(s) per core'

Thread(s) per core:  2


In [8]:
!lscpu | grep "L3 cache" 

L3 cache:            56320K


In [9]:
!lscpu | grep "MHz"

CPU MHz:             2200.000


In [10]:
!free -h --si | awk  '/Mem:/{print $2}'

26G


In [0]:
# !cat /proc/meminfo

In [0]:
#!cat /proc/cpuinfo

In [12]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# os.mkdir("/content/drive/My Drive/wotr_colab") 
os.chdir("/content/drive/My Drive/wotr_colab") 

## Download datasets
1. PlantDoc Object Detection
2. PlantDoc Dataset
2. PlantVillage

In [0]:
# !git clone https://github.com/pratikkayal/PlantDoc-Object-Detection-Dataset.git
# !git clone https://github.com/pratikkayal/PlantDoc-Dataset.git
# !git clone https://github.com/spMohanty/PlantVillage-Dataset.git

## Data Preprocessing
1. Creating Cropped PlantDoc

In [15]:
import os
from tqdm import tqdm

import cv2
import pandas as pd


def crop_image(img_path, output_path, ymin, ymax, xmin, xmax):
    img = cv2.imread(img_path)
    try:
        crop_img = img[ymin:ymax, xmin:xmax]
        cv2.imwrite(output_path, crop_img)
    except:
        pass


def create_label_folders(path, labels):
    for label in labels:
        label_path = os.path.join(path, label)
        if not os.path.exists(label_path):
            os.mkdir(label_path)


def preprocess_plantdoc(df, labels, output_path, source_data_path):
    create_label_folders(output_path, labels)
    for idx, row in tqdm(df.iterrows(), total=df.shape[0], desc=f"preparing '{output_path}'"):
        out_path = os.path.join(output_path, row['class'], str(idx)+".jpg")
        image_path = os.path.join(source_data_path, row['filename'])
        if not os.path.exists(out_path):
            crop_image(image_path, out_path, row['ymin'], row['ymax'], row['xmin'], row['xmax'])


def create_cropped_plantdoc(output_path, source_data_path):
    train_df = pd.read_csv(os.path.join(source_data_path, "train_labels.csv"))
    test_df = pd.read_csv(os.path.join(source_data_path, "test_labels.csv"))

    output_path = os.path.join(output_path, "cropped_plantdoc")
    train_output_path = os.path.join(output_path, "TRAIN")
    test_output_path = os.path.join(output_path, "TEST")
    os.mkdir(output_path) if not os.path.exists(output_path) else ""
    os.mkdir(train_output_path) if not os.path.exists(train_output_path) else ""
    os.mkdir(test_output_path) if not os.path.exists(test_output_path) else ""
    labels = train_df['class'].unique()

    preprocess_plantdoc(train_df, labels, train_output_path, os.path.join(source_data_path, "TRAIN"))
    preprocess_plantdoc(test_df, labels, test_output_path, os.path.join(source_data_path, "TEST"))



create_cropped_plantdoc(output_path="", source_data_path="PlantDoc-Object-Detection-Dataset")

preparing 'cropped_plantdoc/TEST': 100%|██████████| 452/452 [00:10<00:00, 45.10it/s]


2. Train Test Split on 29 class PlantVillage

In [0]:
# import shutil
# import os
# import numpy as np

# def get_files_from_folder(path):

#     files = os.listdir(path)
#     return np.asarray(files)

# def split(path_to_data, path_to_test_data, train_ratio):
#     # get dirs
#     _, dirs, _ = next(os.walk(path_to_data))

#     # calculates how many train data per class
#     data_counter_per_class = np.zeros((len(dirs)))
#     for i in range(len(dirs)):
#         path = os.path.join(path_to_data, dirs[i])
#         files = get_files_from_folder(path)
#         data_counter_per_class[i] = len(files)
#     test_counter = np.round(data_counter_per_class * (1 - train_ratio))

#     # transfers files
#     for i in range(len(dirs)):
#         path_to_original = os.path.join(path_to_data, dirs[i])
#         path_to_save = os.path.join(path_to_test_data, dirs[i])

#         #creates dir
#         if not os.path.exists(path_to_save):
#             os.makedirs(path_to_save)
#         files = get_files_from_folder(path_to_original)
#         # moves data
#         for j in range(int(test_counter[i])):
#             dst = os.path.join(path_to_save, files[j])
#             src = os.path.join(path_to_original, files[j])
#             shutil.move(src, dst)

# split("pvd_subset", "pvd_subset_test", 0.8)

## Model Training:<br>
1. Load data
2. Define architecture
3. Train

Load Data

In [17]:
from sklearn import metrics
import torch
import torchvision
import torchvision.models as models
import torch.nn as nn
import torchvision.transforms as transforms
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)


def load_dataset(data_path: str, data_augment=False):
    data_path = data_path
    transforms_ = [transforms.Resize((100, 100))]
    transforms_aug = [transforms.RandomHorizontalFlip(p=0.5),
                      transforms.RandomRotation(25),
                      transforms.RandomResizedCrop(size=100, scale=(0.8, 1.0))] + transforms_
                  
    transformations = transforms_aug if data_augment else transforms_
    
    dataset = torchvision.datasets.ImageFolder(
        root=data_path,
        transform=transforms.Compose(transformations+[transforms.ToTensor(),
                                                      transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]))

    return dataset

def prepare_data(dataset, batch_size=32):
    data_loader = torch.utils.data.DataLoader(
        dataset,
        batch_size=batch_size,
        num_workers=1,
        shuffle=True
    )

    return data_loader


train_df = load_dataset("cropped_plantdoc/TRAIN", data_augment=True)
test_df = load_dataset("cropped_plantdoc/TEST")
pvd_train = load_dataset("pvd_subset_train")
pvd_test = load_dataset("pvd_subset_test")

assert train_df.class_to_idx==test_df.class_to_idx==pvd_train.class_to_idx==pvd_test.class_to_idx, "Class Indexes Mismatch"
#assert train_df.class_to_idx==test_df.class_to_idx, "Class Indexes Mismatch"

train_ldr = prepare_data(train_df)
test_ldr = prepare_data(test_df)
pvtrain_ldr = prepare_data(pvd_train)
pvtest_ldr = prepare_data(pvd_test)


# resnet18 = models.resnet18(pretrained=True)
# alexnet = models.alexnet(pretrained=True)
# squeezenet = models.squeezenet1_0(pretrained=True)
# vgg16 = models.vgg16(pretrained=True)
# densenet = models.densenet161(pretrained=True)
# inception = models.inception_v3(pretrained=True)
# googlenet = models.googlenet(pretrained=True)
# shufflenet = models.shufflenet_v2_x1_0(pretrained=True)
# mobilenet = models.mobilenet_v2(pretrained=True)
# resnext50_32x4d = models.resnext50_32x4d(pretrained=True)
# wide_resnet50_2 = models.wide_resnet50_2(pretrained=True)
# mnasnet = models.mnasnet1_0(pretrained=True)


cuda:0


Display a sample image from C-PD train, C-PD test, PVD train and PVD test respectively

In [0]:
# trainiter = iter(train_ldr)
# images, labels = next(trainiter)

# testiter = iter(test_ldr)
# images2, labels2 = next(testiter)

# pvtr_iter = iter(pvtrain_ldr)
# images3, labels3 = next(pvtr_iter)

# pvts_iter = iter(pvtest_ldr)
# images4, labels4 = next(pvts_iter)

In [0]:
# imshow(images[0])
# print(train_df.classes[labels[0].item()])

# imshow(images2[0])
# print(test_df.classes[labels2[0].item()])

# imshow(images3[0])
# print(pvd_train.classes[labels3[0].item()])

# imshow(images4[0])
# print(pvd_test.classes[labels4[0].item()])

Define architecture

In [0]:
# # Finetuning VGG16 architecture 

# model = models.vgg16(pretrained=True)
# #vgg_custom.features = nn.Sequential(*[vgg_custom.features[i] for i in range(30)])
# #model.avgpool = nn.AdaptiveAvgPool2d(output_size=(3, 3))         #Changed output_size to (3,3) which is the dimension since we have 100x100 sized images
# model.avgpool = nn.AdaptiveAvgPool2d(output_size=(3, 3))         #Changed output_size to (3,3) which is the dimension since we have 100x100 sized images
# model.classifier[0] = nn.Linear(4608, 4096)                      #Changed input length of the first fully connected to layer due to our input images being 100x100 instead of 224x224
# model.classifier[6] = nn.Linear(4096, len(train_df.classes))     #Changed size of last layer to be according to number of classes in our use case
# model.to(device)

In [21]:
models.vgg16()

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

For sliding window approach:

In [0]:
import torch
import torch.nn as nn
#from .utils import load_state_dict_from_url
from torch.hub import load_state_dict_from_url


__all__ = [
    'VGG', 'vgg11', 'vgg11_bn', 'vgg13', 'vgg13_bn', 'vgg16', 'vgg16_bn',
    'vgg19_bn', 'vgg19',
]


model_urls = {
    'vgg11': 'https://download.pytorch.org/models/vgg11-bbd30ac9.pth',
    'vgg13': 'https://download.pytorch.org/models/vgg13-c768596a.pth',
    'vgg16': 'https://download.pytorch.org/models/vgg16-397923af.pth',
    'vgg19': 'https://download.pytorch.org/models/vgg19-dcbb9e9d.pth',
    'vgg11_bn': 'https://download.pytorch.org/models/vgg11_bn-6002323d.pth',
    'vgg13_bn': 'https://download.pytorch.org/models/vgg13_bn-abd245e5.pth',
    'vgg16_bn': 'https://download.pytorch.org/models/vgg16_bn-6c64b313.pth',
    'vgg19_bn': 'https://download.pytorch.org/models/vgg19_bn-c79401a0.pth',
}


class VGG(nn.Module):

    def __init__(self, features, num_classes=1000, init_weights=True):
        super(VGG, self).__init__()
        self.features = features
        # below attributes are custom
        # self.avgpool = nn.AdaptiveAvgPool2d((7, 7))
        self.classifier2 = nn.Sequential(
            nn.Conv2d(512, 4096, (3,3)),
            nn.ReLU(True),
            nn.Dropout(p=0.5, inplace=False),
            nn.Conv2d(4096, 4096, (1,1)),
            nn.ReLU(True),
            nn.Dropout(p=0.5, inplace=False),
            nn.Conv2d(4096, 29, (1,1))
        )
        if init_weights:
            self._initialize_weights()

    def forward(self, x):
        x = self.features(x)
        # x = self.avgpool(x) custom
        # x = torch.flatten(x, 1) # custom
        x = self.classifier2(x)
        return x.squeeze()

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.constant_(m.bias, 0)


def make_layers(cfg, batch_norm=False):
    layers = []
    in_channels = 3
    for v in cfg:
        if v == 'M':
            layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
        else:
            conv2d = nn.Conv2d(in_channels, v, kernel_size=3, padding=1)
            if batch_norm:
                layers += [conv2d, nn.BatchNorm2d(v), nn.ReLU(inplace=True)]
            else:
                layers += [conv2d, nn.ReLU(inplace=True)]
            in_channels = v
    return nn.Sequential(*layers)


cfgs = {
    'A': [64, 'M', 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'B': [64, 64, 'M', 128, 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'D': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512, 'M'],
    'E': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 256, 'M', 512, 512, 512, 512, 'M', 512, 512, 512, 512, 'M'],
}


def _vgg(arch, cfg, batch_norm, pretrained, progress, **kwargs):
    if pretrained:
        kwargs['init_weights'] = False
    model = VGG(make_layers(cfgs[cfg], batch_norm=batch_norm), **kwargs)
    if pretrained:
        state_dict = load_state_dict_from_url(model_urls[arch],
                                              progress=progress)
        model.load_state_dict(state_dict, strict=False) #custom: added argument strict=False. The last FC layers are not using pretrained from ImageNet        
    return model


def vgg16(pretrained=False, progress=True, **kwargs):
    r"""VGG 16-layer model (configuration "D")
    `"Very Deep Convolutional Networks For Large-Scale Image Recognition" <https://arxiv.org/pdf/1409.1556.pdf>`_
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    return _vgg('vgg16', 'D', False, pretrained, progress, **kwargs)

In [23]:
model = vgg16(pretrained=True)

model.to(device)

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/checkpoints/vgg16-397923af.pth


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [0]:
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

Gen RAM Free: 24.4 GB  | Proc size: 6.9 GB
GPU RAM Free: 7109MB | Used: 4332MB | Util  38% | Total 11441MB


Train

In [0]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [0]:
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

In [0]:
k = model(inputs)

In [0]:
k.argmax(dim=1)

tensor([26,  1,  1,  1,  1,  1,  1,  1, 19,  1, 20, 26, 26,  5,  1,  1,  1,  1,
         1, 19,  1,  1, 15,  1,  1,  1, 28, 15,  1,  1,  1,  1],
       device='cuda:0')

In [0]:
labels

tensor([27, 17, 17,  5, 15, 27, 24, 22, 12, 12, 22,  8,  7, 24, 19,  1,  5,  9,
         5, 24, 15, 18, 17,  4, 15, 10, 14,  5, 12, 21, 25, 20],
       device='cuda:0')

In [30]:
import time
t = []
n_batches_tr = len(pvtrain_ldr)
for epoch in range(20):  # loop over the dataset multiple times
    acc = []
    tic = time.time()
    for i, data in enumerate(pvtrain_ldr, 0):    #trying training on plantvillage and testing on c-pd now
        # get the inputs; data is a list of [inputs, labels]
        model.train()
        inputs, labels = data[0].to(device), data[1].to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        if i%10==9 and i<n_batches_tr-1:
          model.eval()
          test_sample = next(iter(pvtest_ldr))
          if len(test_sample[1])!=32:
            continue
          test_X_pv, test_y_pv = test_sample[0].to(device), test_sample[1].to(device)
          with torch.no_grad():
            test_out = model(test_X_pv)
            test_out = test_out.argmax(dim=1)
            test_acc = metrics.accuracy_score(test_y_pv.cpu(), test_out.cpu())
            acc.append(test_acc)
            print(f"Test accuracy CPD estimate after {epoch+1} epochs, {i+1} batches: {str(test_acc)}")
    model.eval()
    with torch.no_grad():
      test_acc = np.mean(acc)
      print(f"Test accuracy est. on CPD in epoch {epoch+1}: {str(test_acc)}")
    toc = time.time()
    t.append(toc-tic)

print('Finished Training')

Test accuracy CPD estimate after 1 epochs, 10 batches: 0.15625
Test accuracy CPD estimate after 1 epochs, 20 batches: 0.09375
Test accuracy CPD estimate after 1 epochs, 30 batches: 0.25
Test accuracy CPD estimate after 1 epochs, 40 batches: 0.21875
Test accuracy CPD estimate after 1 epochs, 50 batches: 0.34375
Test accuracy CPD estimate after 1 epochs, 60 batches: 0.40625
Test accuracy CPD estimate after 1 epochs, 70 batches: 0.65625
Test accuracy CPD estimate after 1 epochs, 80 batches: 0.65625
Test accuracy CPD estimate after 1 epochs, 90 batches: 0.59375
Test accuracy CPD estimate after 1 epochs, 100 batches: 0.625
Test accuracy CPD estimate after 1 epochs, 110 batches: 0.65625
Test accuracy CPD estimate after 1 epochs, 120 batches: 0.75
Test accuracy CPD estimate after 1 epochs, 130 batches: 0.84375
Test accuracy CPD estimate after 1 epochs, 140 batches: 0.5625
Test accuracy CPD estimate after 1 epochs, 150 batches: 0.84375
Test accuracy CPD estimate after 1 epochs, 160 batches: 0.

In [31]:
print(t)

[10644.07561826706, 774.2258820533752, 380.05156660079956, 263.0056564807892, 172.54682421684265, 146.6787211894989, 132.62633538246155, 130.51865100860596, 125.35639238357544, 121.79291725158691, 124.9396448135376, 122.91927027702332, 123.35777521133423, 124.2794554233551, 124.31084728240967, 126.49031519889832, 124.08500838279724, 123.32732057571411, 123.8837161064148, 121.47099256515503]


In [25]:
import time
t = []
n_batches_tr = len(train_ldr)
for epoch in range(20):  # loop over the dataset multiple times
    acc = []
    tic = time.time()
    for i, data in enumerate(train_ldr, 0):    #trying training on plantvillage and testing on c-pd now
        # get the inputs; data is a list of [inputs, labels]
        model.train()
        inputs, labels = data[0].to(device), data[1].to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        if i%10==9 and i<n_batches_tr-1:
          model.eval()
          test_sample = next(iter(test_ldr))
          if len(test_sample[1])!=32:
            continue
          test_X_pv, test_y_pv = test_sample[0].to(device), test_sample[1].to(device)
          with torch.no_grad():
            test_out = model(test_X_pv)
            test_out = test_out.argmax(dim=1)
            test_acc = metrics.accuracy_score(test_y_pv.cpu(), test_out.cpu())
            acc.append(test_acc)
            print(f"Test accuracy CPD estimate after {epoch+1} epochs, {i+1} batches: {str(test_acc)}")
    model.eval()
    with torch.no_grad():
      test_acc = np.mean(acc)
      print(f"Test accuracy est. on CPD in epoch {epoch+1}: {str(test_acc)}")
    toc = time.time()
    t.append(toc-tic)

print('Finished Training')

Test accuracy CPD estimate after 1 epochs, 10 batches: 0.0625
Test accuracy CPD estimate after 1 epochs, 20 batches: 0.15625
Test accuracy CPD estimate after 1 epochs, 30 batches: 0.03125
Test accuracy CPD estimate after 1 epochs, 40 batches: 0.125
Test accuracy CPD estimate after 1 epochs, 50 batches: 0.15625
Test accuracy CPD estimate after 1 epochs, 60 batches: 0.21875
Test accuracy CPD estimate after 1 epochs, 70 batches: 0.21875
Test accuracy CPD estimate after 1 epochs, 80 batches: 0.15625
Test accuracy CPD estimate after 1 epochs, 90 batches: 0.28125
Test accuracy CPD estimate after 1 epochs, 100 batches: 0.25
Test accuracy CPD estimate after 1 epochs, 110 batches: 0.34375
Test accuracy CPD estimate after 1 epochs, 120 batches: 0.34375
Test accuracy CPD estimate after 1 epochs, 130 batches: 0.34375
Test accuracy CPD estimate after 1 epochs, 140 batches: 0.375
Test accuracy CPD estimate after 1 epochs, 150 batches: 0.3125
Test accuracy CPD estimate after 1 epochs, 160 batches: 0.

In [26]:
print(t)

[2858.750773668289, 86.13349223136902, 81.29369688034058, 81.9462571144104, 79.07908248901367, 81.10707330703735, 79.07767343521118, 79.07514905929565, 81.14920997619629, 81.13848304748535, 80.81921982765198, 81.72767758369446, 79.78037881851196, 81.34251475334167, 83.2901976108551, 81.11717367172241, 83.37258839607239, 84.75337433815002, 81.09492325782776, 83.71779584884644]


In [27]:
!nvidia-smi -L

GPU 0: Tesla P100-PCIE-16GB (UUID: GPU-6383b2cd-8c9a-e579-1fd5-0603747dbd84)


In [0]:
print(t)

In [0]:
#torch.save(model.state_dict(), "models/vgg16_custom1_cpd.pth")
#torch.save(model.state_dict(), "vgg16_sw_cpd.pth")

In [0]:
torch.empty(1,1,5).view(5,).size()

torch.Size([5])

In [0]:
def load_dataset(data_path: str, data_augment=False):
    data_path = data_path
    transforms_ = [transforms.Resize((224, 224))]
    transforms_aug = [transforms.RandomHorizontalFlip(p=0.5),
                      transforms.RandomRotation(25)] + transforms_
                      #transforms.RandomResizedCrop(size=100, scale=(0.8, 1.0))] + transforms_
                  
    transformations = transforms_aug if data_augment else transforms_
    
    dataset = torchvision.datasets.ImageFolder(
        root=data_path,
        transform=transforms.Compose(transformations+[transforms.ToTensor(),
                                                      transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]))

    return dataset

def prepare_data(dataset, batch_size=32):
    data_loader = torch.utils.data.DataLoader(
        dataset,
        batch_size=batch_size,
        num_workers=1,
        shuffle=True
    )

    return data_loader

In [0]:
model.load_state_dict(torch.load("vgg16_custom1_cpd.pth"))
model.eval()

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [0]:
test_data = load_dataset("Maize (Corn) Single Leaf Clean")

In [0]:
test_df = prepare_data(test_data)

In [0]:
for data in test_df:
  inputs, labels = data[0].to(device), data[1].to(device)
  model_out = model(inputs)


In [0]:
model_out.argmax(dim=1)

tensor([7, 8, 8, 7, 8, 7], device='cuda:0')

In [0]:
train_df.classes[7]

'Corn Gray leaf spot'